## Full parameter search for new LPDC codes

#### Imports

In [4]:
import sys
import io
import numpy as np
import re
import time
import itertools
from numpy.linalg import matrix_power as matrix_power
from numpy.linalg import matrix_rank as matrix_rank


from mip import Model, xsum, minimize, BINARY
from bposd.css import css_code
from concurrent.futures import ThreadPoolExecutor
import multiprocessing

from parallel_functions import process_combination

import logging
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s INFO %(message)s",  # hardcoded INFO level
    datefmt="%Y-%m-%d %H:%M:%S",
    stream=sys.stdout,
)

#### Helper Functions

In [11]:
def get_net_encoding_rate(k, n):
    return k / (2*n)

In [3]:
# computes the minimum Hamming weight of a binary vector x such that
# stab @ x = 0 mod 2
# logicOp @ x = 1 mod 2
# here stab is a binary matrix and logicOp is a binary vector
def distance_test(stab,logicOp):
  # number of qubits
  n = stab.shape[1]
  # number of stabilizers
  m = stab.shape[0]

  # maximum stabilizer weight
  wstab = np.max([np.sum(stab[i,:]) for i in range(m)])
  # weight of the logical operator
  wlog = np.count_nonzero(logicOp)
  # how many slack variables are needed to express orthogonality constraints modulo two
  num_anc_stab = int(np.ceil(np.log2(wstab)))
  num_anc_logical = int(np.ceil(np.log2(wlog)))
  # total number of variables
  num_var = n + m*num_anc_stab + num_anc_logical

  model = Model()
  model.verbose = 0
  x = [model.add_var(var_type=BINARY) for i in range(num_var)]
  model.objective = minimize(xsum(x[i] for i in range(n)))

  # orthogonality to rows of stab constraints
  for row in range(m):
    weight = [0]*num_var
    supp = np.nonzero(stab[row,:])[0]
    for q in supp:
      weight[q] = 1
    cnt = 1
    for q in range(num_anc_stab):
      weight[n + row*num_anc_stab +q] = -(1<<cnt)
      cnt+=1
    model+= xsum(weight[i] * x[i] for i in range(num_var)) == 0

  # odd overlap with logicOp constraint
  supp = np.nonzero(logicOp)[0]
  weight = [0]*num_var
  for q in supp:
    weight[q] = 1
  cnt = 1
  for q in range(num_anc_logical):
      weight[n + m*num_anc_stab +q] = -(1<<cnt)
      cnt+=1
  model+= xsum(weight[i] * x[i] for i in range(num_var)) == 1

  model.optimize()

  opt_val = sum([x[i].x for i in range(n)])

  return int(opt_val)


def distance_test_parallel(stab, logicOp):
    n = stab.shape[1]  # number of qubits
    m = stab.shape[0]  # number of stabilizers

    # Parallel computation of wstab
    with ThreadPoolExecutor() as executor:
        wstab_list = list(executor.map(np.sum, stab))
    wstab = np.max(wstab_list)

    wlog = np.count_nonzero(logicOp)
    num_anc_stab = int(np.ceil(np.log2(wstab)))
    num_anc_logical = int(np.ceil(np.log2(wlog)))
    num_var = n + m * num_anc_stab + num_anc_logical

    model = Model()
    model.verbose = 0
    x = [model.add_var(var_type=BINARY) for i in range(num_var)]
    model.objective = minimize(xsum(x[i] for i in range(n)))

    # Function to prepare and add a constraint for a row of stab
    def add_stab_constraint(row):
        weight = [0] * num_var
        supp = np.nonzero(stab[row, :])[0]
        for q in supp:
            weight[q] = 1
        cnt = 1
        for q in range(num_anc_stab):
            weight[n + row * num_anc_stab + q] = -(1 << cnt)
            cnt += 1
        return xsum(weight[i] * x[i] for i in range(num_var)) == 0

    # Parallel addition of orthogonality constraints
    with ThreadPoolExecutor() as executor:
        constraints = list(executor.map(add_stab_constraint, range(m)))
    for constraint in constraints:
        model += constraint

    # Adding odd overlap with logicOp constraint (not parallelized)
    supp = np.nonzero(logicOp)[0]
    weight = [0] * num_var
    for q in supp:
        weight[q] = 1
    cnt = 1
    for q in range(num_anc_logical):
        weight[n + m * num_anc_stab + q] = -(1 << cnt)
        cnt += 1
    model += xsum(weight[i] * x[i] for i in range(num_var)) == 1

    model.optimize()
    opt_val = sum(x[i].x for i in range(n))

    return int(opt_val)

In [50]:
import numpy as np
from itertools import product
import logging
from typing import Optional

def search_codes_general(
        l_range: range[int], 
        m_range: range[int], 
        weight_range: range[int], 
        power_range_A: range[int], 
        power_range_B: range[int], 
        encoding_rate_threshold: Optional[float],
    ):
    """
    Searching the parameter space for good bicycle codes (BC)

    args:
        - l_range: Range of possible values for parameter l
        - m_range: Range of possible values for parameter m
        - weight_range: Range of code weights (= the total number of summands accumulated for both A and B)
        - power_range_A: Range of possible values for exponents for terms in A (A is a sum over polynomials in x and y)
        - power_range_B: Range of possible values for exponents for terms in B (B is a sum over polynomials in x and y)
        - encoding_rate_threshold (float): the lower bound for codes to be saved for further analysis
    """
    good_configs = []

    try:

        for l, m in product(l_range, m_range):
            I_ell = np.identity(l, dtype=int)
            I_m = np.identity(m, dtype=int)
            x, y = {}, {}

            # Generate base matrices x and y
            for i in range(l):
                x[i] = np.kron(np.roll(I_ell, i, axis=1), I_m)
            for j in range(m):
                y[j] = np.kron(I_ell, np.roll(I_m, j, axis=1))

            # Iterate over weights and distribute them across A and B
            for weight in weight_range:
                for weight_A in range(1, weight):  # Ensure at least one term in A and B # TODO: Could think of also raising to the power of zero leading to identity matrix
                    weight_B = weight - weight_A

                    # Generate all combinations of summands in A and B with their respective weights
                    summands_A = list(product(['x', 'y'], repeat=weight_A))
                    summands_B = list(product(['x', 'y'], repeat=weight_B))

                    for summand_combo_A, summand_combo_B in product(summands_A, summands_B):
                        # Iterate over power ranges for each summand in A and B
                        for powers_A in product(power_range_A, repeat=weight_A):
                            for powers_B in product(power_range_B, repeat=weight_B):
                                A, B = np.zeros((l*m, l*m), dtype=int), np.zeros((l*m, l*m), dtype=int)
                                A_poly_sum, B_poly_sum = '', ''

                                # Construct A with its summands and powers
                                for summand, power in zip(summand_combo_A, powers_A):
                                    matrix = x[power] if summand == 'x' else y[power]
                                    print('A.shape:', A.shape)
                                    print('matrix.shape:', matrix.shape)
                                    A += matrix
                                    A_poly_sum += f"{summand}{power} + "

                                # Construct B with its summands and powers
                                for summand, power in zip(summand_combo_B, powers_B):
                                    matrix = x[power] if summand == 'x' else y[power]
                                    print('B.shape:', B.shape)
                                    print('matrix.shape:', matrix.shape)
                                    B += matrix
                                    B_poly_sum += f"{summand}{power} + "

                                # Remove trailing ' + '
                                A_poly_sum = A_poly_sum.rstrip(' + ')
                                B_poly_sum = B_poly_sum.rstrip(' + ')

                                # Transpose matrices A and B
                                AT = np.transpose(A)
                                BT = np.transpose(B)

                                # Construct matrices hx and hz
                                hx = np.hstack((A, B))
                                hz = np.hstack((BT, AT))

                                # Construct and test the CSS code
                                qcode = css_code(hx, hz)  # Define css_code, assuming it's defined elsewhere
                                if qcode.test():  # Define the test method for qcode
                                    r = get_net_encoding_rate(qcode.K, qcode.N)  # Define get_net_encoding_rate
                                    encoding_rate_threshold = 1/15 if encoding_rate_threshold is None else encoding_rate_threshold
                                    if r > encoding_rate_threshold:  # Check your specific criteria for good configurations
                                        code_config = {
                                            'l': l,
                                            'm': m,
                                            'num_phys_qubits': qcode.N,
                                            'num_log_qubits': qcode.K,
                                            'lz': qcode.lz,
                                            'lx': qcode.lx,
                                            'k': qcode.lz.shape[0], 
                                            'encoding_rate': r,
                                            'A_poly_sum': A_poly_sum,
                                            'B_poly_sum': B_poly_sum
                                        }
                                        good_configs.append(code_config)

    except Exception as e:
        logging.logging('An error happened in the parameter space search.', e)
        
    return good_configs

In [20]:
# Define the specific values for l, m, and weight
l_value = range(6, 7) # only the value 6
m_value = range(6, 7) # only the value 6
weight_value = range(6, 7) # only the value 6

# Define the power ranges for summands in A and B
# Adjust these ranges as per the specific code you're trying to reproduce
power_range_A = range(1, 4)  # Example range, adjust as needed
power_range_B = range(1, 4)  # Example range, adjust as needed

# Call the function with the specific values
# good_configs = search_codes_general(
#     l_range=[l_value], 
#     m_range=[m_value], 
#     weight_range=[weight_value], 
#     power_range_A=power_range_A, 
#     power_range_B=power_range_B
# )

In [23]:
total_iterations = sum(
    (weight - 1) * (2 ** weight) * sum(
        (len(power_range_A) ** weight_A) * (len(power_range_B) ** (weight - weight_A))
        for weight_A in range(1, weight)
    )
    for weight in weight_value
) * len(l_value) * len(m_value)
total_iterations

1600

In [10]:
total_iterations = len(l_value) * len(m_value) * len(weight_value) * ((len(weight_value) - 1) / 2) * len(power_range_A) ** (len(weight_value) - 1) * len(power_range_B) ** (len(weight_value) - 1)
iterations_per_10_percent = total_iterations / 10
current_progress = 0
iteration_counter = 0

logging.info("Starting search_codes_general. Total iterations: %d", total_iterations)
total_iterations


0.0

#### Verify if the code in the paper can be found/reproduced with the code search function

In [2]:
import pickle
with open('codes_no_distance.pickle', 'rb') as file:
    # Load the data from the file
    parallel_data = pickle.load(file)

In [5]:
# Look for the first code in the paper of IBM
criteria = {'ell': 6, 'm': 6, 'n_phys_qubits': 72, 'n_log_qubits': 12}

filtered_list = [d for d in parallel_data if all(d.get(key) == value for key, value in criteria.items())]

def find_matching_config(configs, a_poly, b_poly):
    for config in configs:
        if config.get('A_poly_sum') == a_poly and config.get('B_poly_sum') == b_poly:
            print('Found code in the paper!')
            return config  # Return the matching config if found
    raise ValueError('Code in the paper could not be found! Verify the code search function!')  # Return None if no match is found

paper_config = find_matching_config(parallel_data, 'x3 + y1 + y2', 'y3 + x1 + x2')
paper_config

ValueError: Code in the paper could not be found! Verify the code search function!

### Parallelization of code search

In [7]:
def build_code(max_ell, max_m, num_summands_a, num_summands_b):
    all_configs = []
    args_list = []

    for ell in range(6, max_ell + 1):
        for m in range(6, max_m + 1):
            for fixed_x_exponent_a in range(1, 4):
                for fixed_y_exponent_b in range(1, 4):
                    args_list.append((ell, m, fixed_x_exponent_a, fixed_y_exponent_b, num_summands_a, num_summands_b))

    # Determine the number of processes to use
    num_processes = min(multiprocessing.cpu_count(), len(args_list))

    with multiprocessing.Pool(processes=num_processes) as pool:
        results = pool.map(process_combination, args_list)

    # Flatten the list of lists
    for result in results:
        all_configs.extend(result)

    return all_configs

In [8]:
### Surpress the print statements of the qcode.test() function
# Save the current stdout so we can restore it later
original_stdout = sys.stdout
# Redirect stdout to a dummy StringIO object
sys.stdout = io.StringIO()

good_configs_parallel = build_code(
    max_ell=6, 
    max_m=6, 
    num_summands_a=3, 
    num_summands_b=3
)

In [10]:
filtered_list = [d for d in good_configs_parallel if all(d.get(key) == value for key, value in criteria.items())]

def find_matching_config(configs, a_poly, b_poly):
    for config in configs:
        if config.get('A_poly_sum') == a_poly and config.get('B_poly_sum') == b_poly:
            print('Found code in the paper!')
            return config  # Return the matching config if found
    raise ValueError('Code in the paper could not be found! Verify the code search function!')  # Return None if no match is found

paper_config = find_matching_config(good_configs_parallel, 'x3 + y1 + y2', 'y3 + x1 + x2')
paper_config

{'ell': 6,
 'm': 6,
 'n_phys_qubits': 72,
 'n_log_qubits': 12,
 'encoding_rate': 0.08333333333333333,
 'A_poly_sum': 'x3 + y1 + y2',
 'B_poly_sum': 'y3 + x1 + x2'}

#### Rebuild a code from a configuration

In [11]:
import numpy as np
import re  # For parsing the polynomial strings

def rebuild_code(config):
    # Extract configuration details
    ell = config['ell']
    m = config['m']
    A_poly_sum = config['A_poly_sum']
    B_poly_sum = config['B_poly_sum']

    # Define cyclic shift matrices
    I_ell = np.identity(ell, dtype=int)
    I_m = np.identity(m, dtype=int)
    x, y = {}, {}
    for i in range(ell):
        x[i] = np.kron(np.roll(I_ell, i, axis=1), I_m)
    for i in range(m):
        y[i] = np.kron(I_ell, np.roll(I_m, i, axis=1))

    # Initialize A and B matrices
    A = np.zeros((ell*m, ell*m), dtype=int)
    B = np.zeros((ell*m, ell*m), dtype=int)

    # Parse A_poly_sum and B_poly_sum to construct A and B
    for term in re.findall(r'([xy]\d+)', A_poly_sum):
        idx = int(term[1:]) - 1  # Convert term to 0-based index
        if term.startswith('x') and idx < ell:
            A += x[idx]
        elif term.startswith('y') and (idx < m):
            A += y[idx]

    for term in re.findall(r'([xy]\d+)', B_poly_sum):
        idx = int(term[1:]) - 1  # Convert term to 0-based index
        if term.startswith('x') and idx < ell:
            B += x[idx]
        elif term.startswith('y') and (idx < m):
            B += y[idx]

    # Ensure matrices are binary
    A %= 2
    B %= 2

    # Transpose matrices A and B
    AT = np.transpose(A)
    BT = np.transpose(B)

    # Construct matrices hx and hz
    hx = np.hstack((A, B))
    hz = np.hstack((BT, AT))

    qcode = css_code(hx, hz)

    # Construct and return the CSS code using hx and hz
    return {
        'qcode': qcode,
        'hx': hx,
        'hz': hz,
    }

In [12]:
code_info = rebuild_code(paper_config)
print('CSS code rebuilt based on the configuration.')